Exports a basic link table (no DB base, it's all just variables) to a mapping sheet, suitable for import to the transforms tables.  

Things currently not done:  
 - complex hash keys    (this is a basic bitch script)

In [65]:
import json
import os
import sqlite3

from pathlib import Path
from datetime import datetime

conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "../export_output_files/TXT/"
output_filename = "databricks_vault_TabDelm_" + time_string + ".txt"

In [66]:
# Function: Collect a new KeyPhrase from the DB for the hash
def GetNewKeyPhrase():
  sql_query = "SELECT NewKeyPhrase FROM NewKeyPhrase;";
  cur.execute(sql_query)
  row = cur.fetchone()
  NewKeyPhrase = row[0]
  #print(NewKeyPhrase)
  if NewKeyPhrase == '' : # if the view returns a blank row because it randomly selected a used ID
    NewKeyPhrase = GetNewKeyPhrase()
  sql_query = "INSERT INTO AppUsedId (AppUsedId) VALUES (?);";
  cur.execute(sql_query, [NewKeyPhrase])
  conn.commit()
  return NewKeyPhrase

In [67]:
# source table 1
target_entity_name1 = ''  # eg if the link was l_Customer_Property_owner, 'Customer' would go here
source1_server_name = ''
source1_database_name = ''
source1_schema_name = 'dbo'
source1_table_name = ''
source1_key_column_name = ''
source1_join_column_name = ''

# source table 2
target_entity_name2 = ''  # eg if the link was l_Customer_Property_owner, 'Property' would go here
source2_server_name = ''
source2_database_name = ''
source2_schema_name = 'dbo'
source2_table_name = ''
source2_key_column_name = ''
source2_join_column_name = ''

record_source = '1' # '1' for source 1, '2' for source 2, something else for something else

# this should be the identifiers for the vault that you are putting into
target_server_name = 'poc'
target_database_name = 'pocipc'
target_schema_name = 'rv'

target_link_suffix = ''  # eg if the link was l_Customer_Property_owner, 'owner' would go here

# Prefix for DV Core field names to avoid table clashes
core_field_prefix = ''

In [68]:
# Convert the identifiers into single strings for the queries
target_entity_name = target_entity_name1 + '_' + target_entity_name2
if (target_link_suffix != ''):
  target_entity_name = target_entity_name + '_' + target_link_suffix

target_link_name = 'l_' + target_entity_name
target_link_hashkey_name = target_entity_name + '_HashKey'

entity1_hashkey_name = target_entity_name1 + 'HashKey'
entity1_hashkey_identifier = source1_server_name + '.' + source1_database_name + '.' + source1_schema_name + '.' + source1_table_name + '.' + source1_key_column_name
entity1_join_identifier = source1_table_name + '.' + source1_join_column_name

entity2_hashkey_name = target_entity_name2 + 'HashKey'
entity2_hashkey_identifier = source2_server_name + '.' + source2_database_name + '.' + source2_schema_name + '.' + source2_table_name + '.' + source2_key_column_name
entity2_join_identifier = source2_table_name + '.' + source2_join_column_name


In [69]:
# Set link hash logic
link_hash_string = "HASH(" + entity1_hashkey_identifier + " + '|' + " + entity2_hashkey_identifier + ")"
#print(link_hash_string)

In [70]:
# Set join string
join_string = entity1_join_identifier + " = " + entity2_join_identifier
#print(join_string)

In [71]:
# Set record source
if (record_source == '1'):
  record_source = source1_server_name + '.' + source1_database_name + '.' + source1_schema_name + '.' + source1_table_name
elif (record_source == '2'):
  record_source = source2_server_name + '.' + source2_database_name + '.' + source2_schema_name + '.' + source2_table_name  
else:
  record_source = record_source
#print(record_source)  

In [72]:
# Get a new ID for the hashdiff group
hash_group_id = GetNewKeyPhrase()
#print(hash_group_id)

In [73]:
# Set dict list of link
link_fields = [
  # source 1, has the hash logic and the join
  {
      'SERVER_NAME_SOURCE': source1_server_name
    , 'DATABASE_NAME_SOURCE': source1_database_name
    , 'SCHEMA_NAME_SOURCE': source1_schema_name
    , 'TABLE_NAME_SOURCE': source1_table_name
    , 'COLUMN_NAME_SOURCE': source1_key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': target_link_hashkey_name
    , 'ORDINAL_POSITION': 1
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': None
    , 'TRANSFORM_GROUP': hash_group_id
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': link_hash_string
    , 'JOIN_CLAUSE': join_string
  }
  # source 2
  , {
      'SERVER_NAME_SOURCE': source2_server_name
    , 'DATABASE_NAME_SOURCE': source2_database_name
    , 'SCHEMA_NAME_SOURCE': source2_schema_name
    , 'TABLE_NAME_SOURCE': source2_table_name
    , 'COLUMN_NAME_SOURCE': source2_key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': target_link_hashkey_name
    , 'ORDINAL_POSITION': 1
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': None
    , 'TRANSFORM_GROUP': hash_group_id
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
    , 'JOIN_CLAUSE': None
  }
  # LoadDate
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': core_field_prefix + 'LoadDate'
    , 'ORDINAL_POSITION': 2
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'TIMESTAMP_NTZ'
    , 'TRANSFORM_LOGIC': 'DV load datetime'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
    , 'JOIN_CLAUSE': None
  }
  # RecordSource
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': core_field_prefix + 'RecordSource'
    , 'ORDINAL_POSITION': 3
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': "''" + record_source + "'"
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
    , 'JOIN_CLAUSE': None
  }
  # ProcessID
  , {
      'SERVER_NAME_SOURCE': None
    , 'DATABASE_NAME_SOURCE': None
    , 'SCHEMA_NAME_SOURCE': None
    , 'TABLE_NAME_SOURCE': None
    , 'COLUMN_NAME_SOURCE': None
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': core_field_prefix + 'ProcessID'
    , 'ORDINAL_POSITION': 4
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'System Process ID'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
    , 'JOIN_CLAUSE': None
  }
  # Source 1 hash key
  , {
      'SERVER_NAME_SOURCE': source1_server_name
    , 'DATABASE_NAME_SOURCE': source1_database_name
    , 'SCHEMA_NAME_SOURCE': source1_schema_name
    , 'TABLE_NAME_SOURCE': source1_table_name
    , 'COLUMN_NAME_SOURCE': source1_key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': entity1_hashkey_name
    , 'ORDINAL_POSITION': 5
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'HASH(' + entity1_hashkey_identifier + ')'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
    , 'JOIN_CLAUSE': None
  }
  # Source 2 hash key
  , {
      'SERVER_NAME_SOURCE': source2_server_name
    , 'DATABASE_NAME_SOURCE': source2_database_name
    , 'SCHEMA_NAME_SOURCE': source2_schema_name
    , 'TABLE_NAME_SOURCE': source2_table_name
    , 'COLUMN_NAME_SOURCE': source2_key_column_name
    , 'SERVER_NAME_TARGET': target_server_name
    , 'DATABASE_NAME_TARGET': target_database_name
    , 'SCHEMA_NAME_TARGET': target_schema_name
    , 'TABLE_NAME_TARGET': target_link_name
    , 'COLUMN_NAME_TARGET': entity2_hashkey_name
    , 'ORDINAL_POSITION': 6
    , 'IS_NULLABLE': 'NO'
    , 'DATA_TYPE': 'STRING'
    , 'TRANSFORM_LOGIC': 'HASH(' + entity2_hashkey_identifier + ')'
    , 'TRANSFORM_GROUP': None
    , 'TRANSFORM_GROUP_SUFFIX': None
    , 'TRANSFORM_GROUP_LOGIC': None
    , 'JOIN_CLAUSE': None
  }
]

# Append the lists
result_dict_list = link_fields


#for row_dict in result_dict_list:
#  print(row_dict)

In [74]:
# Write to a tab-separated text file
with open(os.path.join(output_path, output_filename), "w") as f:
  for row_dict in result_dict_list:
    line = "\t".join(str(row_dict[key]) for key in row_dict)
    f.write(line + "\n")